In [1]:
%load_ext memory_profiler

# Short tutorial: context-specific GRN inference from 10x multiome dataset
This is a [Dictys](https://github.com/pinellolab/dictys) tutorial to reconstruct and analyze context-specific GRNs from a [10x multiome dataset on PBMC](https://www.10xgenomics.com/resources/datasets/pbmc-from-a-healthy-donor-granulocytes-removed-through-cell-sorting-10-k-1-standard-2-0-0). For a full version with details of individual steps, see tutorial [full-multiome](https://github.com/pinellolab/dictys/tree/master/doc/tutorials/full-multiome).

Before running, you need to **change the runtime configurations** for your machine at [2. Network inference configuration](#2.-Network-inference-configuration) and [3. Network inference](#3.-Network-inference).

If you face any issues or need any assistance, see [FAQ](https://github.com/pinellolab/dictys#faq) and [Issues](https://github.com/pinellolab/dictys#issues).

By using this notebook, you imply that you have already accepted the terms of use and privacy policy on the [10x dataset webpage](https://www.10xgenomics.com/resources/datasets/pbmc-from-a-healthy-donor-granulocytes-removed-through-cell-sorting-10-k-1-standard-2-0-0). The dataset summary webpage is also available [here](https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_10k/pbmc_granulocyte_sorted_10k_web_summary.html).



## 1. Preparation of individual input files in `data` folder
After data processing with CellRanger, you need the following files:
* RNA read count matrix ("Filtered feature barcode matrix MEX (DIR)")
* ATAC reads ("ATAC Position-sorted alignments (BAM)")
* Optionally or from other softwares: cell clustering (inside "Secondary analysis outputs (DIR)")


In [2]:
import os

# 更新 PATH 环境变量，添加 Conda 环境的 bin 目录
os.environ['PATH'] = os.path.expanduser('~/.conda/envs/dictys/bin') + ':' + os.environ['PATH']

# 确保路径已更新
print(os.getenv('PATH'))

# 尝试运行命令
#os.system('dictys_helper expression_mtx.py -h')

/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/.conda/envs/dictys/bin:/opt/gridware/apps/gcc/R/4.3.2/bin:/opt/gridware/compilers/gcc/11.2.0/bin:/opt/gridware2/apps/binapps/anaconda3/2019.07/bin:/opt/gridware2/apps/binapps/anaconda3/2019.07/condabin:/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/.local/bin:/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/bin:/usr/share/Modules/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin


In [3]:
# Removes CPU usage limit by some jupyter versions
import os
os.environ['KMP_AFFINITY'] = ''
#Create input data folder
current_path = os.getcwd()
print(current_path)


/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys


In [9]:
current_path = "/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k"
os.chdir(current_path )

In [14]:
os.getcwd()

'/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k'

In [13]:
%%bash
cd ./data

### expression.tsv.gz
Read count matrix of RNA-profiled cells in compressed tsv format. Downloaded and converted from "Filtered feature barcode matrix MEX (DIR)".

In [15]:
%%bash
cd ./data
# Download expression data in mtx.gz format
wget -q -o /dev/null https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.tar.gz
tar xf pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.tar.gz
rm pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.tar.gz
# Convert from mtx.gz to tsv.gz format using helper script `expression_mtx.py`.
dictys_helper expression_mtx.py filtered_feature_bc_matrix expression.tsv.gz
rm -Rf filtered_feature_bc_matrix


### bams
This folder contains one bam file for each cell with chromatin accessibility measurement. File name should be cell name. Downloaded and converted from "ATAC Position-sorted alignments (BAM)".
* **This step can take hours or even over a day**
* The default setting will need ~30GB of memory for this dataset. Specify a lower `--buffer_size 1000` or smaller for `dictys_helper split_bam.sh` if you have less memory.


In [16]:
%%bash
set -eo pipefail
cd ./data
# Download chromatin accessibility reads in bam format
wget --debug --progress=bar:force:noscroll -O bams.bam https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_atac_possorted_bam.bam


Setting --progress (progress) to bar:force:noscroll
Setting --output-document (outputdocument) to bams.bam
DEBUG output created by Wget 1.21.4 on linux-gnu.

Reading HSTS entries from /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/.wget-hsts
URI encoding = ‘UTF-8’
--2024-08-01 15:03:26--  https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_atac_possorted_bam.bam
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:ad, ...
Caching cf.10xgenomics.com => 104.18.1.173 104.18.0.173 2606:4700::6812:ad 2606:4700::6812:1ad
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
Created socket 4.
Releasing 0x0000558dffbeaf70 (new refcount 1).
Initiating SSL handshake.
Handshake successful; connected socket 4 to SSL handle 0x0000558dffbf69d0
certificate:
  subject: CN=10xgenomics.com
  issuer:  CN=WE1,O=Google Trust Services,C=US
X509 certificate succes

In [17]:
%%bash
set -eo pipefail
cd ./data
/usr/bin/time -v dictys_helper split_bam.sh bams.bam bams --section "CB:Z:" --ref_expression expression.tsv.gz
rm bams.bam

	Command being timed: "dictys_helper split_bam.sh bams.bam bams --section CB:Z: --ref_expression expression.tsv.gz"
	User time (seconds): 1097.66
	System time (seconds): 187.20
	Percent of CPU this job got: 60%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 35:15.91
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 8777140
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 18
	Minor (reclaiming a frame) page faults: 85689530
	Voluntary context switches: 17729036
	Involuntary context switches: 5392
	Swaps: 0
	File system inputs: 122180616
	File system outputs: 128438344
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


### subsets & subsets.txt
* subsets.txt: Names of cell subsets. For each cell subset, a GRN is reconstructed.
* subsets: Folder containing one subfolder for each cell subset as in `subsets.txt`. Each subfolder contains two files:
    - names_rna.txt: Names of cells that belong to this subset and have transcriptome measurement
    - names_atac.txt: Names of cells that belong to this subset and have chromatin accessibility measurement
    - For joint measurements of RNA and ATAC, these two files should be identical in every folder.
    
Option 1: download the clustering from 10x "Secondary analysis outputs (DIR)":

In [18]:
%%bash
cd ./data
wget -q -o /dev/null https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_3k/pbmc_granulocyte_sorted_3k_analysis.tar.gz
#Extract cell names for each cluster
tar xf pbmc_granulocyte_sorted_3k_analysis.tar.gz 
mv analysis/clustering/gex/graphclust/clusters.csv clusters.csv
rm -Rf pbmc_granulocyte_sorted_3k_analysis.tar.gz analysis


Option 2: cluster with other softwares (here [scanpy](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html) below).

**Note**:
* The cell below can be hidden if you use a renderer other than jupyter (e.g. github).
* If you don't have scanpy installed, the kernel would automatically restart to finish scanpy installation. Please continue by running this cell again for clustering.

#Installs scanpy if not present
a=!pip show scanpy &> /dev/null; echo $?
a=''.join(a)
if a=='0':
	print('Scanpy found. Continuing notebook execution.')
elif a=='1':
	a=!pip install 'scanpy[leiden]'; echo $?
	a=''.join(a)
	if a!='0':
		raise RuntimeError('Installing scanpy failed.')
	print('Installing scanpy completed. Restarting kernel. Please continue from here.')
	import os
	os._exit(00)
else:
	raise ValueError('Unknown return for: pip show scanpy')

#Clustering with scanpy
import numpy as np
import pandas as pd
import scanpy as sc
adata = sc.read_10x_mtx('../data/filtered_feature_bc_matrix/',var_names='gene_symbols',cache=False)
adata.var_names_make_unique()
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 20, :]
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.leiden(adata)
adata.obs.leiden.to_csv('../data/clusters.csv',header=['Cluster'],index=True,index_label='Barcode')


Finally, reformat clusters.csv for input:

In [19]:
%%bash
cd ./data
subsets="$(tail -n +2 clusters.csv | awk -F , '{print $2}' | sort -u)"
echo "$subsets" | awk '{print "Subset"$1}' > subsets.txt
for x in $subsets; do
	mkdir -p "subsets/Subset$x"
	grep ",$x"'$' clusters.csv | awk -F , '{print $1}' > "subsets/Subset$x/names_rna.txt"
	# RNA and ATAC barcodes are the same for joint quantifications
	cp "subsets/Subset$x/names_rna.txt" "subsets/Subset$x/names_atac.txt"
done
rm clusters.csv


### Other files
Motif, reference genome, gene transcriptional start site

In [20]:
%%bash
cd ./data

# Motifs (file motifs.motif)
# Option 1: from HOCOMOCO (https://hocomoco11.autosome.org/)
wget -q -o /dev/null -O motifs.motif 'https://hocomoco11.autosome.org/final_bundle/hocomoco11/full/HUMAN/mono/HOCOMOCOv11_full_HUMAN_mono_homer_format_0.0001.motif'
# Option 2: from HOMER
#dictys_helper motif_homer.sh > motifs.motif
# Option 3: provide your custom motifs



In [21]:
!head -n 18 ./data/motifs.motif

>dKhGCGTGh	AHR_HUMAN.H11MO.0.B	3.3775000000000004
0.262728374765856	0.1227600511842322	0.362725638699551	0.25178593535036087
0.07633328991810645	0.08258130543118362	0.22593295481662123	0.6151524498340887
0.14450570038747923	0.28392173880411337	0.13815442099009081	0.4334181398183167
0.023935814057894068	0.016203821748029118	0.9253278681170539	0.03453249607702277
0.007919544273173793	0.953597675415874	0.017308392078009837	0.021174388232942286
0.02956192959210962	0.012890110758086997	0.9474192747166682	0.010128684933135217
0.007919544273173797	0.029561929592109615	0.012337825593096645	0.9501807005416201
0.007919544273173793	0.007919544273173793	0.9762413671804787	0.007919544273173793
0.27886589130660366	0.4285328543459993	0.10955683916661985	0.18304441518077724
>hnnGGWWnddWWGGdbWh	AIRE_HUMAN.H11MO.0.C	5.64711
0.38551919443239085	0.2604245534178759	0.1353299124033618	0.21872633974637148
0.18745267949274294	0.18745267949274294	0.14575446582123766	0.4793401751932764
0.14575446582123777	0.145

In [23]:
%%bash
cd ./data
# Reference genome (folder genome)
# Note: You need the same reference genome version with chromatin accessibility reads
# Option 1: download genome from HOMER
dictys_helper genome_homer.sh hg38 genome
# Option 2: provide your custom genome



Process is interrupted.


In [22]:
%%bash
ls -h1s ./data/genome | head

total 4.3G
113K annotations
 64K chrom.sizes
3.7G genome.fa
4.2M hg38.aug
 51M hg38.basic.annotation
258M hg38.full.annotation
272K hg38.miRNA
233M hg38.repeats
 16M hg38.rna


In [29]:
%%bash
cd ./data
# Bed file for TSS (file gene.bed)
# Download gtf file from ensembl
#wget -q -o /dev/null -O gene.gtf.gz http://ftp.ensembl.org/pub/release-107/gtf/homo_sapiens/Homo_sapiens.GRCh38.107.gtf.gz
gunzip gene.gtf.gz
# Convert to bed
dictys_helper gene_gtf.sh gene.gtf gene.bed
rm gene.gtf


In [23]:
!head ./data/gene.bed

chr1	11869	14409	DDX11L1	.	+
chr1	14404	29570	WASH7P	.	-
chr1	17369	17436	MIR6859-1	.	-
chr1	29554	31109	MIR1302-2HG	.	+
chr1	30366	30503	MIR1302-2	.	+
chr1	34554	36081	FAM138A	.	-
chr1	52473	53312	OR4G4P	.	+
chr1	57598	64116	OR4G11P	.	+
chr1	65419	71585	OR4F5	.	+
chr1	131025	134836	CICP27	.	+


### Clean up

In [24]:
!rm -Rf ./data/filtered_feature_bc_matrix


## 2. Network inference configuration
### Generate configurations
Please adjust them for your own machine and dataset

In [25]:
pwd

'/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k'

In [26]:
%%bash
# Generate configuration template
rm -Rf ./makefiles
mkdir ./makefiles
cd ./makefiles
dictys_helper makefile_template.sh common.mk config.mk env_none.mk static.mk

# Update configurations, such as:
# DEVICE: pytorch device, e.g. cpu, cuda:0. If you do not have a GPU, use 'cpu' and expect LONG computing time.
# GENOME_MACS2: effective genome size for macs2. See https://deeptools.readthedocs.io/en/develop/content/feature/effectiveGenomeSize.html
# JOINT: whether dataset is joint profiling of RNA and ATAC.
# Other configurations include quality control thresholds, number of threads in each job, number of hidden confounders, etc.
# They can be obtained in the full-multiome tutorial.
dictys_helper makefile_update.py ../makefiles/config.mk '{"DEVICE": "cpu", "GENOME_MACS2": "hs", "JOINT": "1"}'

In [27]:
!cat ./makefiles/config.mk


# Lingfei Wang, 2022. All rights reserved.
#This file contains parameters for whole run and individual steps to be edited for your dataset
#This file should be edited to configure the run
#This file should NOT be directly used for any run with `makefile -f` 

############################################################
# Run environment settings
############################################################
#Which environment to use, corresponding to env_$(ENVMODE).mk file
ENVMODE=none
#Maximum number of CPU threads for each job
#This is only nominated and passed through to other softwares without any guarantee.
NTH=4
#Device name for pyro/pytorch
#Note: cuda devices other than cuda:0 could be incompatible with singularity environment
DEVICE=cpu

############################################################
# Dataset settings
############################################################

#Genome size for Macs2, accept shortcuts like mm & hs
GENOME_MACS2=hs
#Whether dataset is joint profili

### Validate input data

In [28]:
%%bash
/usr/bin/time -v cd . &&  /usr/bin/time -v dictys_helper makefile_check.py

	Command being timed: "cd ."
	User time (seconds): 0.00
	System time (seconds): 0.00
	Percent of CPU this job got: 50%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.00
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 3328
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 189
	Voluntary context switches: 8
	Involuntary context switches: 0
	Swaps: 0
	File system inputs: 8
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


Joint profile: True
Found 2714 cells with RNA profile
Found 24036 genes with RNA profile
Found 2714 cells with ATAC profile
Found 769 motifs
Found 678 TFs
Found 461 TFs in current dataset
Missing 217 TFs in current dataset: ANDR,AP2A,AP2B,AP2C,AP2D,ARI3A,ARI5B,ATF6A,BARH1,BARH2,BC11A,BHA15,BHE22,BHE23,BHE40,BHE41,BMAL1,BRAC,BSH,COE1,COT1,COT2,CR3L1,CR3L2,ERR1,ERR2,ERR3,EVI1,GCR,HEN1,HMBX1,HME1,HME2,HNF6,HTF4,HXA1,HXA10,HXA11,HXA13,HXA2,HXA5,HXA7,HXA9,HXB1,HXB13,HXB2,HXB3,HXB4,HXB6,HXB7,HXB8,HXC10,HXC11,HXC12,HXC13,HXC6,HXC8,HXC9,HXD10,HXD11,HXD12,HXD13,HXD3,HXD4,HXD8,HXD9,ITF2,KAISO,MCR,MGAP,MLXPL,MYBA,MYBB,NDF1,NDF2,NF2L1,NF2L2,NFAC1,NFAC2,NFAC3,NFAC4,NGN2,NKX21,NKX22,NKX23,NKX25,NKX28,NKX31,NKX32,NKX61,NKX62,ONEC2,ONEC3,OZF,P53,P5F1B,P63,P73,PEBB,PHX2A,PHX2B,PIT1,PKNX1,PLAL1,PO2F1,PO2F2,PO2F3,PO3F1,PO3F2,PO3F3,PO3F4,PO4F1,PO4F2,PO4F3,PO5F1,PO6F1,PO6F2,PRD14,PRGR,RHXF1,RORG,RX,SMCA1,SMCA5,SRBP1,SRBP2,STA5A,STA5B,STF1,SUH,TF2LX,TF65,TF7L1,TF7L2,TFE2,THA,THA11,THB,TWST1,TYY1,TYY2,UBIP1,

	Command being timed: "dictys_helper makefile_check.py"
	User time (seconds): 17.75
	System time (seconds): 5.53
	Percent of CPU this job got: 54%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:42.88
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 7923800
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 6
	Minor (reclaiming a frame) page faults: 4280792
	Voluntary context switches: 65301
	Involuntary context switches: 791
	Swaps: 0
	File system inputs: 6408392
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


## 3. Network inference
* **This step can take a day or longer**. You are strongly suggested to use a GPU. If it takes over a day, see [FAQ](https://github.com/pinellolab/dictys#faq) to improve speed.
* Please adjust `-j n_jobs` for your machine. The total thread count is n_jobs*n_threads. By default n_threads=4.


In [29]:
import os
print(os.cpu_count())

32


In [30]:
%memit

peak memory: 74.64 MiB, increment: 0.00 MiB


In [31]:
%%bash
cd .; 
/usr/bin/time -v dictys_helper network_inference.sh -j 32 -J 1 static 

mkdir -p tmp_static/Subset1/
cp data/subsets/Subset1/names_rna.txt tmp_static/Subset1/names_rna.txt
mkdir -p tmp_static/Subset2/
mkdir -p tmp_static/Subset3/
cp data/subsets/Subset2/names_rna.txt tmp_static/Subset2/names_rna.txt
cp data/subsets/Subset3/names_rna.txt tmp_static/Subset3/names_rna.txt
mkdir -p tmp_static/Subset4/
mkdir -p tmp_static/Subset5/
cp data/subsets/Subset4/names_rna.txt tmp_static/Subset4/names_rna.txt
cp data/subsets/Subset5/names_rna.txt tmp_static/Subset5/names_rna.txt
mkdir -p tmp_static/Subset6/
cp data/subsets/Subset6/names_rna.txt tmp_static/Subset6/names_rna.txt
mkdir -p tmp_static/Subset7/
cp data/subsets/Subset7/names_rna.txt tmp_static/Subset7/names_rna.txt
mkdir -p tmp_static/Subset8/
cp data/subsets/Subset8/names_rna.txt tmp_static/Subset8/names_rna.txt
mkdir -p tmp_static/Subset9/
cp data/subsets/Subset9/names_rna.txt tmp_static/Subset9/names_rna.txt
cp tmp_static/Subset1/names_rna.txt tmp_static/Subset1/names_atac0.txt
OPENBLAS_NUM_THREADS=1 NUMEXP

OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  preproc selects_atac  tmp_static/Subset4/expression.tsv.gz tmp_static/Subset4/names_atac0.txt tmp_static/Subset4/names_atac.txt
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  preproc selects_atac  tmp_static/Subset1/expression.tsv.gz tmp_static/Subset1/names_atac0.txt tmp_static/Subset1/names_atac.txt
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  preproc selects_atac  tmp_static/Subset2/expression.tsv.gz tmp_static/Subset2/names_atac0.txt tmp_static/Subset2/names_atac.txt
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  chromatin macs2 --nth 4 tmp_static/S

DEBUG @ Thu, 01 Aug 2024 17:09:24: access pq hash for 8985502 times 
INFO  @ Thu, 01 Aug 2024 17:09:24: #3 Call peaks for each chromosome... 
INFO  @ Thu, 01 Aug 2024 17:09:45: #4 Write output xls file... 04_peaks.xls 
INFO  @ Thu, 01 Aug 2024 17:09:45: #4 Write peak in narrowPeak format file... 04_peaks.narrowPeak 
INFO  @ Thu, 01 Aug 2024 17:09:45: #4 Write summits bed file... 04_summits.bed 
INFO  @ Thu, 01 Aug 2024 17:09:45: Done! 

OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  chromatin wellington --nth 4 tmp_static/Subset9/reads.bam tmp_static/Subset9/reads.bai tmp_static/Subset9/peaks.bed tmp_static/Subset9/footprints.bed
[bam_sort_core] merging from 1 files and 4 in-memory blocks...
INFO  @ Thu, 01 Aug 2024 17:09:37: 
# Command line: callpeak -t /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/tmp_static/Subset6/reads.bam -n 04 -g hs --nomodel --sh

INFO  @ Thu, 01 Aug 2024 17:10:06: * Input file is gzipped. 
INFO  @ Thu, 01 Aug 2024 17:10:09:  1000000 
INFO  @ Thu, 01 Aug 2024 17:10:12:  2000000 
INFO  @ Thu, 01 Aug 2024 17:10:15:  3000000 
INFO  @ Thu, 01 Aug 2024 17:10:18:  4000000 
INFO  @ Thu, 01 Aug 2024 17:10:21:  5000000 
INFO  @ Thu, 01 Aug 2024 17:10:22: 5346533 reads have been read. 
INFO  @ Thu, 01 Aug 2024 17:10:22: #1 tag size is determined as 49 bps 
INFO  @ Thu, 01 Aug 2024 17:10:22: #1 tag size = 49.0 
INFO  @ Thu, 01 Aug 2024 17:10:22: #1  total tags in treatment: 5346533 
INFO  @ Thu, 01 Aug 2024 17:10:22: #1 finished! 
INFO  @ Thu, 01 Aug 2024 17:10:22: #2 Build Peak Model... 
INFO  @ Thu, 01 Aug 2024 17:10:22: #2 Skipped... 
INFO  @ Thu, 01 Aug 2024 17:10:22: #2 Use 150 as fragment length 
INFO  @ Thu, 01 Aug 2024 17:10:22: #2 Sequencing ends will be shifted towards 5' by 75 bp(s) 
INFO  @ Thu, 01 Aug 2024 17:10:22: #3 Call peaks... 
INFO  @ Thu, 01 Aug 2024 17:10:22: #3 Going to call summits inside each peak 

INFO  @ Thu, 01 Aug 2024 17:13:27: Done! 

OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  chromatin wellington --nth 4 tmp_static/Subset1/reads.bam tmp_static/Subset1/reads.bai tmp_static/Subset1/peaks.bed tmp_static/Subset1/footprints.bed
[bam_sort_core] merging from 2 files and 4 in-memory blocks...
INFO  @ Thu, 01 Aug 2024 17:12:08: 
# Command line: callpeak -t /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/tmp_static/Subset4/reads.bam -n 04 -g hs --nomodel --shift -75 --extsize 150 --keep-dup all --verbose 4 --call-summits -q 0.05
# ARGUMENTS LIST:
# name = 04
# format = AUTO
# ChIP-seq file = ['/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/tmp_static/Subset4/reads.bam']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap bet

...

OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network normalize --nth 4 tmp_static/Subset8/net_weight.tsv.gz tmp_static/Subset8/net_meanvar.tsv.gz tmp_static/Subset8/net_covfactor.tsv.gz tmp_static/Subset8/net_nweight.tsv.gz
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network indirect --nth 4 --fi_meanvar tmp_static/Subset8/net_meanvar.tsv.gz tmp_static/Subset8/net_weight.tsv.gz tmp_static/Subset8/net_covfactor.tsv.gz tmp_static/Subset8/net_iweight.tsv.gz
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network normalize --nth 4 tmp_static/Subset8/net_iweight.tsv.gz tmp_static/Subset8/net_meanvar.tsv.gz tmp_static/Subset8/net_covfactor.tsv.gz tmp_static/Subset8/net_inweight.tsv.gz
Reading BED Fi

	Extracting 649 sequences from chr20
	Extracting 289 sequences from chr21
	Extracting 538 sequences from chr22
	Extracting 1189 sequences from chr3
	Extracting 682 sequences from chr4
	Extracting 948 sequences from chr5
	Extracting 1139 sequences from chr6
	Extracting 996 sequences from chr7
	Extracting 801 sequences from chr8
	Extracting 921 sequences from chr9
	Extracting 456 sequences from chrX
	Extracting 3 sequences from chrY


	Reading input files...
	21674 total sequences read
	769 motifs loaded
	Finding instances of 769 motif(s)
	|0%                                    50%                                  100%|
	Cleaning up tmp files...


	Position file = 14-reform-split/aaaaa
	Genome = /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/data/genome
	Output Directory = 15-motifscan/aaaaa
	Using actual sizes of regions (-size given)
	Fragment size set to given
	Will use repeat masked sequences
	Will find motif(s) in /mnt/iusers01/fatpou01/bmh01/msc-h

	Peak/BED file conversion summary:
		BED/Header formatted lines: 25001
		peakfile formatted lines: 0

	Peak File Statistics:
		Total Peaks: 25001
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Background fragment size set to 19 (avg size of targets)
	Background files for 19 bp fragments found.
	Custom genome sequence directory: /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/data/genome

	Extracting sequences from file: /mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/data/genome/genome.fa
	Looking for peak sequences in a single file (/mnt/iusers01/fatpou01/bmh01/msc-healthdatasci-2023-2024/z89953zj/models/dictys3k/data/genome/genome.fa)
	Extracting 2490 sequences from chr1
	Extracting 1106 sequences from chr10
	Extracting 1315

OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network normalize --nth 4 tmp_static/Subset6/net_weight.tsv.gz tmp_static/Subset6/net_meanvar.tsv.gz tmp_static/Subset6/net_covfactor.tsv.gz tmp_static/Subset6/net_nweight.tsv.gz
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network indirect --nth 4 --fi_meanvar tmp_static/Subset6/net_meanvar.tsv.gz tmp_static/Subset6/net_weight.tsv.gz tmp_static/Subset6/net_covfactor.tsv.gz tmp_static/Subset6/net_iweight.tsv.gz
OPENBLAS_NUM_THREADS=1 NUMEXPR_NUM_THREADS=1 MKL_NUM_THREADS=1 OPENBLAS_MAX_THREADS=1 NUMEXPR_MAX_THREADS=1 MKL_MAX_THREADS=1 python3 -m dictys  network normalize --nth 4 tmp_static/Subset6/net_iweight.tsv.gz tmp_static/Subset6/net_meanvar.tsv.gz tmp_static/Subset6/net_covfactor.tsv.gz tmp_static/Subset6/net_inweight.tsv.gz
OPENBLAS_NUM_T

	Command being timed: "dictys_helper network_inference.sh -j 32 -J 1 static"
	User time (seconds): 298030.63
	System time (seconds): 11741.26
	Percent of CPU this job got: 1650%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 5:12:53
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 3537696
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 2645
	Minor (reclaiming a frame) page faults: 821627917
	Voluntary context switches: 89403623
	Involuntary context switches: 50477882
	Swaps: 0
	File system inputs: 39619872
	File system outputs: 60216152
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [32]:
%memit

peak memory: 74.64 MiB, increment: 0.00 MiB
